# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *

In [2]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load and edit files

LAM output analysis.

simu 1500km et simu 1000km.

## Sims

In [7]:
LAM_1500_40_dir='../../../JZ_simu_outputs/LAM/LAM_1500_40_sim224'
LAM_1500_60_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60_sim206'
smallLAM_dir='../../../JZ_simu_outputs/LAM/noirr_2010_2022'

In [8]:
lon_min=-18
lon_max=11
lat_min=29
lat_max=52
# lon_min=-13
# lon_max=6
# lat_min=32
# lat_max=49
date_min = '2010-01-01'
# date_max = '2012-12-31'
date_max = '2013-06-30'

In [ ]:
#sim
# filename = '{}/ATM/TS*.nc'.format(largeLAM_dir)
filename = '{}/ATM/MO/*.nc'.format(LAM_1500_40_dir)

sim_40_0 = xr.open_mfdataset(filename)
sim_40_0.attrs['name'] = '1500_nbp40'
sim_40= sim_40_0.rename({'time_counter':'time'})
sim_40= sim_40.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))
sim_40= sim_40.sel(time=slice(date_min, date_max))

sim_40['evap'] = sim_40['evap'] *3600 * 24
sim_40['evap'].attrs['units'] = 'mm/d'

sim_40['precip'] = sim_40['precip'] *3600 * 24
sim_40['precip'].attrs['units'] = 'mm/d'

sim_40['fluxsens']= -sim_40['sens']

sim_40['netrad'] = sim_40['LWdnSFC'] - sim_40['LWupSFC'] + sim_40['SWdnSFC'] - sim_40['SWupSFC']
sim_40['netrad'].attrs['units'] = 'W/m2'

sim_40['P - E'] = sim_40['precip'] - sim_40['evap']
sim_40['P - E'].attrs['units'] = 'mm/d'

# sim_40['uq'] = sim_40['uq'] #* 3600 * 24 #* sim_40['aire']
sim_40['uq'].attrs['units'] = 'kg/m/s'
# sim_40['vq'] = sim_40['vq'] #* 3600 * 24 #* sim_40['aire']
sim_40['vq'].attrs['units'] = 'kg/m/s'

sim_40['calcWind'] = np.sqrt(sim_40['u10m']**2 + sim_40['v10m']**2)
sim_40['calcWind'].attrs['units'] = 'm/s'

sim_40

In [ ]:
filename = '{}/ATM/MO/*.nc'.format(LAM_1500_60_dir)

sim_60_0 = xr.open_mfdataset(filename)
sim_60_0.attrs['name'] = '1500_nbp60'
sim_60= sim_60_0.rename({'time_counter':'time'})
sim_60= sim_60.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))
sim_60= sim_60.sel(time=slice(date_min, date_max))

sim_60['evap'] = sim_60['evap'] *3600 * 24
sim_60['evap'].attrs['units'] = 'mm/d'
sim_60['precip'] = sim_60['precip'] *3600 * 24
sim_60['precip'].attrs['units'] = 'mm/d'
sim_60['fluxsens']= -sim_60['sens']
sim_60['netrad'] = sim_60['LWdnSFC'] - sim_60['LWupSFC'] + sim_60['SWdnSFC'] - sim_60['SWupSFC']
sim_60['netrad'].attrs['units'] = 'W/m2'
sim_60['P - E'] = sim_60['precip'] - sim_60['evap']
sim_60['P - E'].attrs['units'] = 'mm/d'
# sim_60['uq'] = sim_60['uq'] #* 3600 * 24 #* sim_60['aire']
sim_60['uq'].attrs['units'] = 'kg/m/s'
# sim_60['vq'] = sim_60['vq'] #* 3600 * 24 #* sim_60['aire']
sim_60['vq'].attrs['units'] = 'kg/m/s'
sim_60['calcWind'] = np.sqrt(sim_60['u10m']**2 + sim_60['v10m']**2)
sim_60['calcWind'].attrs['units'] = 'm/s'

sim_60

In [ ]:
#smaller LAM (original)
filename = '{}/ATM/TS*.nc'.format(smallLAM_dir)
# filename = '{}/ATM/MO/*.nc'.format(smallLAM_dir)

sim0small = xr.open_mfdataset(filename)
sim0small.attrs['name'] = '1000km'
small = sim0small.rename({'time_counter':'time'})
small = small.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))
small = small.sel(time=slice(date_min, date_max))

small['evap'] = small['evap'] *3600 * 24
small['evap'].attrs['units'] = 'mm/d'

small['precip'] = small['precip'] *3600 * 24
small['precip'].attrs['units'] = 'mm/d'

small['fluxsens']= -small['sens']

small['netrad'] = small['LWdnSFC'] - small['LWupSFC'] + small['SWdnSFC'] - small['SWupSFC']
small['netrad'].attrs['units'] = 'W/m2'

small['P - E'] = small['precip'] - small['evap']
small['P - E'].attrs['units'] = 'mm/d'

small


## Obs

In [ ]:
#open ERA5 file
# filename = '../../../obs/ERA5_eval_2010_2022.nc'
filename = '../../../obs/ERA5_large_eval.nc'
era = xr.open_mfdataset(filename)
era.attrs['name'] = 'ERA5'
#define or edit vars
#name ERA5 vars like in LMDZOR outputs
dict =  {
        'longitude':'lon',
        'latitude':'lat',
        'e':'evap',
        'ro':'totrunoff',
        'tp':'precip',
        'u10':'u10m',
        'v10':'v10m',
        'p71.162' : 'uq',
        'p72.162' : 'vq',
        'sshf' : 'fluxsens',
        'slhf' : 'fluxlat',
        'si10' : 'sfcWind',
        'ssrd' : 'SWdnSFC',
        'strd' : 'LWdnSFC',
        'ssr' : 'SWnetSFC',
        'str' : 'LWnetSFC',
        }
era = era.rename(dict)
era=era.sel(time=slice(date_min, date_max))
era = era.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max, lat_min))

#make evap positive in era
era['evap'] = -era['evap'] * 1000
era['evap'].attrs['units'] = 'mm/d'
era['precip'] = era['precip']*1000
era['precip'].attrs['units'] = 'mm/d'
# era['cldt'] = era['tcc']*100
# era['cldt'].attrs['units'] = '%'

era['P - E'] = era['precip'] - era['evap']
era['P - E'].attrs['units'] = 'mm/d'

era['calcWind'] = np.sqrt(era['u10m']**2 + era['v10m']**2)
era['calcWind'].attrs['units'] = 'm/s'
   
#convert 4 radiation variables from J/m2 to W/m2
era['SWdnSFC'] = era['SWdnSFC'] / (3600 *24)
era['SWdnSFC'].attrs['units'] = 'W/m2'
era['LWdnSFC'] = era['LWdnSFC'] / (3600 *24)
era['LWdnSFC'].attrs['units'] = 'W/m2'
era['SWnetSFC'] = era['SWnetSFC'] / (3600 *24)
era['SWnetSFC'].attrs['units'] = 'W/m2'
era['LWnetSFC'] = era['LWnetSFC'] / (3600 *24)
era['LWnetSFC'].attrs['units'] = 'W/m2'
#add up vars and netrad
era['LWupSFC'] = - era['LWnetSFC'] + era['LWdnSFC']
era['LWupSFC'].attrs['units'] = 'W/m2'
era['SWupSFC'] = - era['SWnetSFC'] + era['SWdnSFC']
era['SWupSFC'].attrs['units'] = 'W/m2'
era['netrad'] = era['LWnetSFC'] + era['SWnetSFC']
era['netrad'].attrs['units'] = 'W/m2'

era


In [ ]:
#gleam
filename='../../../obs/evap/E_1980-2022_GLEAM_v3.8a_MO.nc'
gleam0=xr.open_dataset(filename)
gleam0.attrs['name']='GLEAM'
gleam = gleam0.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max,lat_min))
#adapt period
gleam=gleam.sel(time=slice(date_min, date_max))
#name gleam vars like in LMDZOR outputs
dict =  {
        'E':'evap'
        }
gleam = gleam.rename(dict)
#convert mm/month to mm/day
gleam['evap']=gleam['evap']/30
gleam['evap'].attrs['units']='mm/d'
gleam

In [ ]:
#FluxCom
filename='../../..//obs/FluxCom/ensemble_rs-_meteo_ALL/monthly/*.RS_METEO.EBC-*.MLM-ALL.METEO-ALL.720_360.monthly.*.nc'
fluxcom0=xr.open_mfdataset(filename)
fluxcom=fluxcom0.sel(lon=slice(lon_min,lon_max+0.25),lat=slice(lat_max,lat_min))
fluxcom=fluxcom.sel(time=slice(date_min, date_max))

fluxcom.attrs["name"]='FluxCom'

fluxcom['evap'] = fluxcom['LE'] * 0.408 # To convert from MJ/m2/d to mm/d
fluxcom['evap'].attrs['units'] = 'mm/d'

fluxcom['fluxlat'] = fluxcom['LE'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['fluxlat'].attrs['units'] = 'W/m²'

fluxcom['fluxsens'] = fluxcom['H'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['fluxsens'].attrs['units'] = 'W/m²'

fluxcom['netrad'] = fluxcom['Rn'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['netrad'].attrs['units'] = 'W/m²'
fluxcom

In [ ]:
#gpcc
filename='../../../obs/precips/precip.mon.total.0.25x0.25.v2020.nc'
gpcc0=xr.open_mfdataset(filename)
gpcc0.attrs['name'] = 'GPCC'
gpcc=gpcc0.sel(time=slice(date_min, date_max))
gpcc['lon'] = ((gpcc['lon'] + 180) % 360) - 180
gpcc1 = gpcc.sortby('lon')

# gpcc=gpcc.sel(lon=slice(lon_min,lon_max+0.25),lat=slice(lat_min,lat_max))
gpcc=gpcc1.sel(lon=slice(lon_min,lon_max+0.25),lat=slice(lat_max,lat_min))

# gpcc1=gpcc.sel(lon=slice(347,360),lat=slice(lat_max,lat_min))
# gpcc2=gpcc.sel(lon=slice(0,lon_max+0.25),lat=slice(lat_max,lat_min))
# gpcc=xr.combine_by_coords([gpcc1, gpcc2])
# gpcc = gpcc.where(gpcc['lon'] < lon_max+0.25, drop=True)

# gpcc['lon']=gpcc['lon']-180.0
# gpcc=gpcc.sel(lon=slice(lon_min,lon_max+0.25))

gpcc['precip'] = gpcc['precip'] / 30 #convert to mm/d
gpcc['precip'].attrs['units'] = 'mm/d'
gpcc

In [ ]:
#open tqruv file
filename='../../../obs/TQRUV_850_2010_2022_monthly.nc'
tqruv = xr.open_dataset(filename)
tqruv.attrs['name'] = 'ERA5'
tqruv = tqruv.sel(longitude=slice(lon_min,lon_max),latitude=slice(lat_max,lat_min))
tqruv = tqruv.sel(time=slice(date_min, date_max))
#rename longitude and latitude to lon and lat
tqruv = tqruv.rename({'longitude':'lon','latitude':'lat'})

#rename all variables to add 850 in name
tqruv['u850'] = tqruv['u']
tqruv['v850'] = tqruv['v']
tqruv['q850'] = tqruv['q']
tqruv['t850'] = tqruv['t']
tqruv['r850'] = tqruv['r']
tqruv

# Masking, interpolation

In [17]:
# Select months
# months=[7]

# orc = orc.sel(time=orc['time.month'].isin(months))
# orcirr = orcirr.sel(time=orcirr['time.month'].isin(months))

# sim = sim_40.sel(time=sim_40['time.month'].isin(months))
# small = small.sel(time=small['time.month'].isin(months))

# era = era.sel(time=era['time.month'].isin(months))
# fluxcom=fluxcom.sel(time=fluxcom['time.month'].isin(months))

In [18]:
#continental fraction mask (LMDZ outputs)
con_mask=sim_60['contfracATM']>0.9

#create a masks of data points where irrig_frac >5%
# irr_mask = (orcirr['irrigmap_dyn']/orcirr['Areas'])>0.05
irr_mask = con_mask

ip_mask=polygon_to_mask(sim_60, iberic_peninsula)
# ip_mask_orc=polygon_to_mask(orc, iberic_peninsula)

In [19]:
sim_small_i60=small.interp_like(sim_60)
sim_40_i60=sim_40.interp_like(sim_60)

In [ ]:
cont_sim_60=sim_60.where(con_mask)
cont_sim_40=sim_40.where(con_mask)
cont_small=sim_small.where(con_mask)

ip_sim_60 = sim_60.where(ip_mask['mask'], drop=False).where(con_mask)
ip_sim_40 = sim_40_i60.where(ip_mask['mask'], drop=False).where(con_mask)
ip_small = sim_small.where(ip_mask['mask'], drop=False).where(con_mask)

In [23]:
sim_40_era = sim_40.interp_like(era)
sim_40_gpcc = sim_40.interp_like(gpcc)
sim_40_gleam = sim_40.interp_like(gleam)
sim_40_fluxcom = sim_40.interp_like(fluxcom)
sim_40_tqruv = sim_40.interp_like(tqruv)

sim_60_era = sim_60.interp_like(era)
sim_60_gpcc = sim_60.interp_like(gpcc)
sim_60_gleam = sim_60.interp_like(gleam)
sim_60_fluxcom = sim_60.interp_like(fluxcom)
sim_60_tqruv = sim_60.interp_like(tqruv)

small_era = small.interp_like(era)
small_gpcc = small.interp_like(gpcc)
small_gleam = small.interp_like(gleam)
small_fluxcom = small.interp_like(fluxcom)
small_tqruv = small.interp_like(tqruv)

In [ ]:
ip_sim_40_era = ip_sim_40.interp_like(era)
ip_sim_40_gpcc = ip_sim_40.interp_like(gpcc)
ip_sim_40_gleam = ip_sim_40.interp_like(gleam)
ip_sim_40_fluxcom = ip_sim_40.interp_like(fluxcom)
ip_sim_40_tqruv = ip_sim_40.interp_like(tqruv)

ip_sim_60_era = ip_sim_60.interp_like(era)
ip_sim_60_gpcc = ip_sim_60.interp_like(gpcc)
ip_sim_60_gleam = ip_sim_60.interp_like(gleam)
ip_sim_60_fluxcom = ip_sim_60.interp_like(fluxcom)
ip_sim_60_tqruv = ip_sim_60.interp_like(tqruv)

ip_small_era = ip_small.interp_like(era)
ip_small_gpcc = ip_small.interp_like(gpcc)
ip_small_gleam = ip_small.interp_like(gleam)
ip_small_fluxcom = ip_small.interp_like(fluxcom)
ip_small_tqruv = ip_small.interp_like(tqruv)

In [113]:
# era
int_era=era.interp_like(sim_60)
cont_era=int_era.where(con_mask)
ip_era = cont_era.where(ip_mask['mask'], drop=False)
# orc_era=era.interp_like(orc)

In [114]:
#gleam
int_gleam=gleam.interp_like(sim_60)
cont_gleam=gleam.interp_like(sim_60).where(con_mask)
ip_gleam = cont_gleam.where(ip_mask['mask'], drop=False)
# irr_era=era.interp_like(orc).where(irr_mask)
# irr_gleam=gleam.interp_like(orc).where(irr_mask)

In [115]:
#fluxcom
int_fluxcom=fluxcom.interp_like(sim_60)
cont_fluxcom=fluxcom.interp_like(sim_60).where(con_mask)
ip_fluxcom = cont_fluxcom.where(ip_mask['mask'], drop=False)
# orc_fluxcom=fluxcom.interp_like(orc)

In [116]:
#gpcc
int_gpcc=gpcc.interp_like(sim_60)
cont_gpcc=gpcc.interp_like(sim_60).where(con_mask)
ip_gpcc = cont_gpcc.where(ip_mask['mask'], drop=False)

In [117]:
#era5 tqruv
int_tqruv = tqruv.interp_like(sim_60)
cont_tqruv = tqruv.interp_like(sim_60).where(con_mask)
ip_tqruv = cont_tqruv.where(ip_mask['mask'], drop=False)

# Maps

In [ ]:
#Display map of var for ds
var='sfcWind'
ds=sim
color_map=reds
max_value=None
min_value=None

map_ave(ds, var, cmap=color_map, vmin=min_value, vmax=max_value, hex=False)

In [ ]:
#Diff map
var='precip'
max_value=None
min_value=None
ds1=sim_60_era
ds2=era
color_map=emb

map_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, cmap=color_map, hex=True, sig=False)

diff=ds1[var]-ds2[var]
title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
map_seasons(diff, cmap=color_map, vmax=max_value, title=title, hex=False)

In [ ]:
#map for 4 seasons
var='precip'
ds1=cont_sim
ds2=cont_era
max_value=2
min_value=None
# min_value=-max_value

diff=ds1[var]-ds2[var]
title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
plotvar=diff

# plotvar=ds1[var]
# title='P-E (mm/d, {})'.format(ds1.attrs['name'])

map_seasons(plotvar, cmap=emb, vmax=max_value, title=title, hex=False)

In [ ]:
# Display 2 maps of var
var = 'precip'
min_value=None
max_value=6
color_map=wet
ds1=era
ds2=sim_60
map_two_ds(ds1, ds2, var, vmin=min_value, figsize=(15,6), vmax=max_value, cmap=color_map, hex=True)

In [ ]:
#Display map of var for ds on restricted area
var='evap'
ds = sim
latmin=36
latmax=36.9
lonmin=-6.4
lonmax=-6

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')
map_plotvar(plotvar, cmap=emb_neutral, vmax=None, title=var)

In [ ]:
#Relative diff map
var='precip'
max_value=None
ds1=cont_fluxcom
ds2=ip_fluxcom
# plt.title('Gross primary production relative difference (%)')
map_rel_diff_ave(ds1, ds2, var, vmax=max_value)

In [ ]:
ds=sim
map_wind(ds, height='10m', scale=50)

In [ ]:
ds=sim_era
map_moisture_transport(ds, scale=1500)

# Time series

In [ ]:
var='fluxsens'

ds1=ip_sim
ds2=ip_small
ds3=ip_era

ds3=ip_fluxcom

ds4=ip_gpcc

# ds4=ip_fluxcom
# ds5=ip_gleam

# ds1=ip_orc
# ds2=ip_orcirr
# ds3=ip_fluxcom_orc

ds_list=[ds1, ds2, ds3]#, ds4]
# ds_list=[ds4]

year_max=2012
time_series_ave(ds_list, var, year_max=year_max, title='{} {}'.format(var, ds1[var].attrs['units']))
seasonal_cycle_ave(ds_list, var, year_max=year_max, title='Seasonal cycle of {} ({})'.format(var, ds1[var].attrs['units']))